# Monitoring and Evaluating NEMO Guardrails apps

TODO: This notebook is in an experimental state.

In [ ]:
# pip uninstall -y trulens_eval
# pip install git+https://github.com/truera/trulens@piotrm/azure_bugfixes#subdirectory=trulens_eval

# trulens_eval notebook dev

%load_ext autoreload
%autoreload 2
from pathlib import Path
from pprint import pprint
import sys

base = Path().cwd()
while not (base / "trulens_eval").exists():
    base = base.parent

print(base)

# If running from github repo, can use this:
sys.path.append(str(base))

# TODO: presently we use some features of a modified version of NeMo-Guardrails.
# Either point path to it or install from branch.
sys.path.append("/Volumes/dev_new/NeMo-Guardrails")

from trulens_eval.keys import check_keys
check_keys(
    "OPENAI_API_KEY",
    "HUGGINGFACE_API_KEY"
)

from trulens_eval import Tru
tru = Tru()
tru.reset_database()

tru.run_dashboard(_dev=base, force=True)

## Rails app setup

The files created below define a configuration of a rails app adapted from
various examples in the NeMo-Guardrails repository.

In [ ]:
%%writefile config.yaml
# Adapted from NeMo-Guardrails/nemoguardrails/examples/bots/abc/config.yml
instructions:
  - type: general
    content: |
      Below is a conversation between a user and a bot called the trulens Bot.
      The bot is designed to answer questions about the trulens_eval python library.
      The bot is knowledgeable about python.
      If the bot does not know the answer to a question, it truthfully says it does not know.

sample_conversation: |
  user "Hi there. Can you help me with some questions I have about trulens?"
    express greeting and ask for assistance
  bot express greeting and confirm and offer assistance
    "Hi there! I'm here to help answer any questions you may have about the trulens. What would you like to know?"

models:
  - type: main
    engine: openai
    model: gpt-3.5-turbo-instruct

#knowledge_base: # no longer works with openai >= 1.0
#  embedding_search_provider:
#    name: default
#    parameters:
#      embedding_engine: openai
#      embedding_model: text-embedding-ada-002

In [ ]:
%%writefile config.co
# Adapted from NeMo-Guardrails/tests/test_configs/with_kb_openai_embeddings/config.co
define user ask capabilities
  "What can you do?"
  "What can you help me with?"
  "tell me what you can do"
  "tell me about you"

define bot inform capabilities
  "I am an AI bot that helps answer questions about trulens_eval."

define flow
  user ask capabilities
  bot inform capabilities

## Rails app instantiation

In [ ]:
from nemoguardrails import LLMRails, RailsConfig

config = RailsConfig.from_path(".")
rails = LLMRails(config)

## Feedback functions

In [ ]:
from trulens_eval.feedback.feedback import RAG_triad
from trulens_eval.feedback.provider import OpenAI
from trulens_eval.schema import Select
from trulens_eval.tru_rails import TruRails

# Initialize provider class
openai = OpenAI()

# select context to be used in feedback. the location of context is app specific.
from trulens_eval.app import App

context = App.select_context(rails)
question = Select.RecordInput
answer = Select.RecordOutput

triad = RAG_triad(
    provider=openai,
    question=question, answer=answer, context=context
)

pprint(triad)

## `TruRails` recorder instantiation

In [ ]:
tru_rails = TruRails(rails, feedbacks=triad.values())

## logged invocation

Using `tru_rails` as a context manager means the invocations of the rail app
will be logged and feedback will be evaluated on the results.

In [ ]:
with tru_rails as recorder:
    response = await rails.generate_async(messages=[{
    "role": "user",
    "content": "Can I use AzureOpenAI to define a provider?"
}])
    
print(response['content'])

### Feedback retrieval

Feedback results can be viewed on the dashboard or retrieved from the record produced by the context mamanger.

In [ ]:
record = recorder.get()

from concurrent.futures import as_completed

for fut in as_completed(record.feedback_results):
    feedback, result = fut.result()
    print(feedback.name, result.result)